In [78]:
import pandas as pd
from connect import PostgreSQL
from tqdm import tqdm
import datetime as dt
from datetime import date
import plotly.graph_objs as go

# Parâmetros

In [79]:
_schema = 'H-1'

In [80]:
_pixels = 720

pixels_width = int((_pixels/9)*16)
pixels_height = int(_pixels)

In [81]:
date_query = date(int((dt.datetime.now().strftime('%Y-%m-%d').split('-')[0])), int((dt.datetime.now().strftime('%Y-%m-%d').split('-')[1])), 1).strftime('%Y-%m-%d')

In [82]:
codigos_kaizen = [('KAIZEN ASA NORTE', '13996'),
                  ('KAIZEN CEILÂNDIA', '16100'),
                  (     'KAIZEN GAMA', '18400'),
                  (  'KAIZEN SOF SUL', '20000')]

In [83]:
codigos_vended = [(    'GARANTIA', '0100'),
                  ('MARIO SERGIO', '0001'),
                  (   'DEVOLUCAO', '0006')]

In [84]:
lista_lojas = (['01',             'CD'],
               ['02',               ''],
               ['03',  '04. ASA NORTE'],
               ['04',  '02. CEILÂNDIA'],
               ['05',       '03. GAMA'],
               ['06',    '01. SOF SUL'],
               ['VO', '05. E-COMMERCE'])

In [85]:
relatorio = 'ACOMPANHAMENTO DE VENDA'

relatorio_lower = relatorio.lower()
relatorio_msg = relatorio_lower.replace(relatorio_lower.split()[0], relatorio_lower.split()[0].title())

In [86]:
caminho_imagem = 'Y:\\COMPRAS\\arquivo temporario analise atualizado\\DEPARTAMENTO COMPRAS\\10. CARLOS\\REPORT'

In [87]:
contatos = [('PrimeiraFoto', 'https://chat.whatsapp.com/LK4LDajkH52BNydoLGGD2d', True),
            ('Dados e Int',  'https://chat.whatsapp.com/JxY14kWarG70MxB2jzb5pc', False),
            ('Dream Team',   'https://chat.whatsapp.com/Hb0KMgYnuFB8coTPlFKTzJ', False),
            ('Report Di',    'https://chat.whatsapp.com/G4p8qdUIUblCNPS80244RQ', False),
            ('API',          'https://chat.whatsapp.com/EBLX0fxYfvuHL7jZ9CZECQ', True),
            ('Teste Python', 'https://chat.whatsapp.com/I4y7EAVfns629NvfoLUtGI', True)]

#

# DataFrames

## Pedidos

In [88]:
df_pedido = PostgreSQL.read_postgres(name_table='pedido', schema=_schema, query=f'WHERE DT_EMISSAO >= \'{date_query}\'')

In [89]:
df_pedido['VALOR_TOT'] = df_pedido['VALOR_TOT'].astype(float)

In [90]:
for (kaizen, cod_kaizen) in tqdm(codigos_kaizen):
    df_pedido = df_pedido[df_pedido['CODCLI'] != cod_kaizen]

100%|██████████| 4/4 [00:00<00:00, 15.71it/s]


In [91]:
for (vendedor, cod_vende) in tqdm(codigos_kaizen):
    df_pedido = df_pedido[df_pedido['CODCLI'] != cod_vende]

100%|██████████| 4/4 [00:00<00:00, 17.63it/s]


In [92]:
df_pedido = df_pedido[df_pedido['TIPPED'] == 'V']

In [93]:
df_pedido = df_pedido[(df_pedido['OBSERVA'].str.contains('REQ') != True) |
                      (df_pedido['OBSERVA2'].str.contains('REQ') != True)]

In [94]:
for i in tqdm(df_pedido.index):
    if df_pedido.loc[i, 'FORMA_PGTO'] == 'M':
        df_pedido.loc[i, 'COD_LOJA'] = 'VO'
    else:
        df_pedido.loc[i, 'COD_LOJA'] = df_pedido.loc[i, 'CD_LOJA']

100%|██████████| 52606/52606 [00:04<00:00, 11567.14it/s]


## Clientes

In [95]:
df_cliente = PostgreSQL.read_postgres(name_table='cliente', schema=_schema)

## Devolução

In [96]:
df_prod_ent = PostgreSQL.read_postgres(name_table='prod_ent', schema=_schema, query=f'WHERE DT_EMISSAO >= \'{date_query}\'')

In [98]:
for (kaizen, cod_kaizen) in tqdm(codigos_kaizen):
    df_prod_ent = df_prod_ent[df_prod_ent['CD_CLIENTE'] != cod_kaizen]

100%|██████████| 4/4 [00:00<00:00, 124.89it/s]


In [99]:
df_prod_ent

,CD_LOJA,NU_NOTA,SG_SERIE,NU_ORIGEM,SG_ORIGEM,IN_CLIFOR,CD_CLIENTE,CD_FORNECE,CD_VENDEDO,CD_PRODUTO,...,PC_FCPDEST,PC_ICMDEST,PC_ICMREME,PC_ICMPART,VL_FCPDEST,VL_ICMDEST,VL_ICMREME,CD_CLIOLD,CD_FOROLD,NU_ITEM
0,06,047137,NE,046604,NE,None,13690,None,None,025083,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,None,None,1
1,06,047152,NE,045754,NE,None,12134,None,None,053813,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,None,None,1
2,06,047156,NE,042421,NE,None,02185,None,None,034228,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,None,None,1
3,06,047193,NE,042873,NE,None,11592,None,None,021910,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,None,None,1
4,06,047204,NE,045625,NE,None,11330,None,None,037981,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,None,None,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6371,04,246204,NE,245484,NE,None,10845,None,None,059994,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,None,None,1
6380,05,118526,NE,112561,NE,None,12958,None,None,038133,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,None,None,1
6381,06,047155,NE,044664,NE,None,12571,None,None,047957,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,None,None,2
6382,06,047191,NE,044354,NE,None,02183,None,None,047779,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,None,None,1
